In [1]:
"""
具体步骤：
# 1.获取数据集
# 2.基本数据处理
# 2.1 缩⼩数据范围
# 2.2 选择时间特征
# 2.3 去掉签到较少的地⽅
# 2.4 确定特征值和⽬标值
# 2.5 分割数据集
# 3.特征⼯程 -- 特征预处理(标准化)
# 4.机器学习 -- knn+cv
# 5.模型评估
"""
import pandas as pd

data=pd.read_csv("./train.csv")

In [2]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [3]:
data.describe()

,row_id,x,y,accuracy,time,place_id
count,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07
mean,1.455901e+07,4.999770e+00,5.001814e+00,8.284912e+01,4.170104e+05,5.493787e+09
std,8.405649e+06,2.857601e+00,2.887505e+00,1.147518e+02,2.311761e+05,2.611088e+09
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000016e+09
25%,7.279505e+06,2.534700e+00,2.496700e+00,2.700000e+01,2.030570e+05,3.222911e+09
50%,1.455901e+07,5.009100e+00,4.988300e+00,6.200000e+01,4.339220e+05,5.518573e+09
75%,2.183852e+07,7.461400e+00,7.510300e+00,7.500000e+01,6.204910e+05,7.764307e+09
max,2.911802e+07,1.000000e+01,1.000000e+01,1.033000e+03,7.862390e+05,9.999932e+09


In [8]:
data.shape

(29118021, 6)

In [7]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# 2.基本数据处理
# 2.1 缩⼩数据范围

In [32]:
partial_data=data.query("x>2.0&x<2.5&y>2.0&y<2.5")
partial_file='partial.xlsx'
partial_data.to_excel(partial_file,index=False)
print(f'data export to {partial_file} successfully')

data export to partial.xlsx successfully


In [11]:
partial_data.shape

(71664, 6)

# 2.2 选择时间特征

In [44]:
partial_data["time"].head()

163     669737
310     234719
658     502343
1368    319822
1627    595084
Name: time, dtype: int64

In [45]:
time=pd.to_datetime(partial_data["time"],unit="s")##脱敏,unit=s是单位秒
time.head()

163    1970-01-08 18:02:17
310    1970-01-03 17:11:59
658    1970-01-06 19:32:23
1368   1970-01-04 16:50:22
1627   1970-01-07 21:18:04
Name: time, dtype: datetime64[ns]

In [18]:
time.hour

AttributeError: 'Series' object has no attribute 'hour'

In [46]:
time=pd.DatetimeIndex(time)
time

DatetimeIndex(['1970-01-08 18:02:17', '1970-01-03 17:11:59',
               '1970-01-06 19:32:23', '1970-01-04 16:50:22',
               '1970-01-07 21:18:04', '1970-01-02 03:14:59',
               '1970-01-07 03:45:16', '1970-01-05 03:28:43',
               '1970-01-01 18:59:03', '1970-01-09 07:50:12',
               ...
               '1970-01-09 20:03:34', '1970-01-08 09:26:50',
               '1970-01-07 04:45:59', '1970-01-07 22:36:18',
               '1970-01-06 23:29:43', '1970-01-03 12:31:26',
               '1970-01-04 15:19:20', '1970-01-01 20:49:14',
               '1970-01-03 09:17:37', '1970-01-02 20:34:43'],
              dtype='datetime64[ns]', name='time', length=69264, freq=None)

In [47]:
time.hour

Int64Index([18, 17, 19, 16, 21,  3,  3,  3, 18,  7,
            ...
            20,  9,  4, 22, 23, 12, 15, 20,  9, 20],
           dtype='int64', name='time', length=69264)

In [48]:
time.day

Int64Index([8, 3, 6, 4, 7, 2, 7, 5, 1, 9,
            ...
            9, 8, 7, 7, 6, 3, 4, 1, 3, 2],
           dtype='int64', name='time', length=69264)

In [49]:
partial_data["hour"]=time.hour
partial_data["day"]=time.day
partial_data["weekday"]=time.weekday

C:\Users\Dell\AppData\Local\Temp\ipykernel_34200\654291739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partial_data["hour"]=time.hour
C:\Users\Dell\AppData\Local\Temp\ipykernel_34200\654291739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partial_data["day"]=time.day
C:\Users\Dell\AppData\Local\Temp\ipykernel_34200\654291739.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [50]:
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
163,163,2.1663,2.3755,84,669737,3869813743,18,8,3
310,310,2.3695,2.2034,3,234719,2636621520,17,3,5
658,658,2.3236,2.1768,66,502343,7877745055,19,6,1
1368,1368,2.2613,2.3392,73,319822,9775192577,16,4,6
1627,1627,2.3331,2.0011,66,595084,6731326909,21,7,2


# 2.3 去掉签到较少的地⽅

In [51]:
place_count=partial_data.groupby("place_id").count()##groupby分组计数
#1006234733是只有一个,1008823061	是有4个以此类推

In [52]:
place_count=place_count[place_count["row_id"]>3]
place_count.head()

,row_id,x,y,accuracy,time,hour,day,weekday
place_id,,,,,,,,
1008823061,4,4,4,4,4,4,4,4
1025585791,21,21,21,21,21,21,21,21
1026507711,220,220,220,220,220,220,220,220
1032417180,10,10,10,10,10,10,10,10
1040557418,123,123,123,123,123,123,123,123


# 2.4 确定特征值和⽬标值

In [53]:
partial_data=partial_data[partial_data["place_id"].isin(place_count.index)]##isin去掉partial_data里面不在place_count的数据

In [55]:
partial_data.shape

(69264, 9)

In [56]:
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
163,163,2.1663,2.3755,84,669737,3869813743,18,8,3
310,310,2.3695,2.2034,3,234719,2636621520,17,3,5
658,658,2.3236,2.1768,66,502343,7877745055,19,6,1
1368,1368,2.2613,2.3392,73,319822,9775192577,16,4,6
1627,1627,2.3331,2.0011,66,595084,6731326909,21,7,2


In [64]:
x=partial_data[["x","y","accuracy","hour","day","weekday"]]##特征值
y=partial_data["place_id"]##目标值

In [65]:
x.head()

,x,y,accuracy,hour,day,weekday
163,2.1663,2.3755,84,18,8,3
310,2.3695,2.2034,3,17,3,5
658,2.3236,2.1768,66,19,6,1
1368,2.2613,2.3392,73,16,4,6
1627,2.3331,2.0011,66,21,7,2


In [66]:
y.head()

163     3869813743
310     2636621520
658     7877745055
1368    9775192577
1627    6731326909
Name: place_id, dtype: int64

In [93]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.25)##训练集：测试集=0.75：0.25,random_state确定个随机数种子，随机状态
x_train.head()

,x,y,accuracy,hour,day,weekday
23474680,2.2915,2.2799,25,21,6,1
27036383,2.4758,2.2941,169,11,1,3
26070820,2.3224,2.1311,67,10,2,4
26783398,2.0444,2.2766,46,2,5,0
1025674,2.4013,2.0436,35,13,5,0


# 3.特征⼯程 -- 特征预处理(标准化)

In [94]:
transfer=StandardScaler()#实例化
x_train=transfer.fit_transform(x_train)  #fit_transform转换
x_test=transfer.fit_transform(x_test)

# 4.机器学习 -- knn+cv

In [95]:
#4.1实例化一个训练器
estimator=KNeighborsClassifier()

##4.2交叉验证，网格搜索实现
param_grid={"n_neighbors":[1,3,5,7,9]}
estimator=GridSearchCV(estimator=estimator,param_grid=param_grid,cv=10,n_jobs=4)##n_jobs表示电脑用多少个CPU

##4.3模型验证
estimator.fit(x_train,y_train)

D:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9]})

In [96]:
##5.模型评估
##5.1准确率输出

score_ret=estimator.score(x_test,y_test)
print("准确率为\n",score_ret)

准确率为
 0.3686186186186186


In [97]:
##5.2预测结果
y_pre=estimator.predict(x_test)
print(y_pre)

[7431930642 7727090889 3958678140 ... 7309958589 3002793420 2579541805]


In [98]:
##5.3其他结果输出
print("最好的模型：\n",estimator.best_estimator_)
print("最好的结果：\n",estimator.best_score_)
print("最好的结果：\n",estimator.cv_results_)

最好的模型：
 KNeighborsClassifier()
最好的结果：
 0.3645376189228483
最好的结果：
 {'mean_fit_time': array([0.06179197, 0.06310301, 0.0602653 , 0.0603184 , 0.05985947]), 'std_fit_time': array([0.00464692, 0.00400323, 0.00231686, 0.00383894, 0.00154308]), 'mean_score_time': array([0.1329901 , 0.1654098 , 0.17020526, 0.17716768, 0.17633691]), 'std_score_time': array([0.00748607, 0.0121627 , 0.00928128, 0.00441078, 0.00992113]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.36381136, 0.35168431, 0.36862368, 0.37208855, 0.36169394]), 'split1_test_score': array([0.35110683, 0.34494706, 0.35399423, 0.35437921, 0.34841193]), 'split2_test_score': array([0.36207892, 0.34975938, 0.36515881, 0.3599615 , 0.36015399]), 'split3_test_score': array([0.3626564 , 0.35

In [ ]:
"""
具体步骤：
# 1.获取数据集
# 2.基本数据处理
# 2.1 缩⼩数据范围
# 2.2 选择时间特征
# 2.3 去掉签到较少的地⽅
# 2.4 确定特征值和⽬标值
# 2.5 分割数据集
# 3.特征⼯程 -- 特征预处理(标准化)
# 4.机器学习 -- knn+cv
# 5.模型评估
"""